In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Num GPUs Available:  1


In [2]:
print(tf.__version__)

2.3.0


In [3]:
from collections import Counter
import numpy as np
import pandas as pd 
import os
import time
import math
import statistics
from datetime import datetime
import random as rn
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm.notebook import tqdm
import sweetviz as sv

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from keras.layers import Conv2D, AvgPool2D, MaxPool2D, Flatten
import keras
from IPython.display import clear_output


from data_set_builder import data_set_builder
from create_lstm_train_test import transform_df_to_LSTM_train_test


In [4]:
def predictions_vs_actual(predictions, y_test):
    try:
        y_test = y_test.reshape(y_test.shape[0],)
        predictions = predictions.reshape(len(predictions),)
    except: 
        print("y_test       shape: {} ".format(y_test.shape))
        print("predictions  shape: {} ".format(predictions.shape))
    df = pd.DataFrame({'Target':y_test ,'Predictions': predictions })
    df['Difference'] = df['Predictions'] - df['Target'] 
    return df


def plot_predictions(input_df):
    results_df = input_df.reset_index()
    fig = px.line(results_df, x="index", y=[col for col in results_df if col not in ['index', 'Difference']], 
                  title="Predictions vs. Target")
    fig.show()
    
    
def plot_predictions_difference(input_df):
    results_df = input_df.reset_index()
    fig = px.line(results_df, x="index", y=[col for col in results_df if col not in ['index', 'Target', 'Predictions']], 
                  title="Predictions - Target ")
    fig.show()
    
def plot_trading_simulation(total_pips_gained, dates_traded):
    
    cumulative_pips_gained = [0]
    for idx,pips in enumerate(total_pips_gained):
        new_val = pips + cumulative_pips_gained[idx]
        cumulative_pips_gained.append(new_val)
        
    print("dates_traded: ", len(dates_traded))
    print("cumulative_pips_gained: ", len(cumulative_pips_gained))
    
    df = pd.DataFrame({'Dates': dates_traded ,'Cumulative Pips Gained':  cumulative_pips_gained })
    fig = px.line(df, x="Dates", y="Cumulative Pips Gained", 
                  title="Cumulative Pips Gained")
    fig.show()
    


In [5]:
def load_LSTM_test_data(model_type, window_size, future_candles, return_dates = False ):
    """
    model_type: LSTM or CNN
    """
    
    test_df  = data_set_builder('EURUSD', [i for i in range(2020,2021)], future_candles)
    train_df = test_df.loc[:120,:] # The train data is not being used!
    
    if return_dates:
        X_train, X_val, X_test, y_train, y_val, y_test, y_test_dates = transform_df_to_LSTM_train_test(train_df, test_df, future_candles, 
                                                                              window_size, return_dates)
    else:
        X_train, X_val, X_test, y_train, y_val, y_test = transform_df_to_LSTM_train_test(train_df, test_df, future_candles, 
                                                                                    window_size)
    
    if model_type == "CNN":
        # CNN specified input shape (only test data)
        X_test  = X_test.reshape(X_test.shape[0] , X_test.shape[1], X_test.shape[2], 1)
    
    if return_dates:
        return X_test, y_test , y_test_dates
    else: 
        return X_test, y_test

In [6]:
   
    
def load_target_and_predictions(model_folder, model_type, return_results = False, return_dates = False):
    """
    model_folder: an int describing model number
    """
    
    global all_models_folder
    
    results_file = all_models_folder + model_folder + "/y_test_y_pred.csv"
    
    with open(all_models_folder + model_folder + "/model_report.txt") as file:
        input_layer_nodes, h_layer_1_nodes, h_layer_2_nodes, h_layer_3_nodes = 0,0,0,0

        for line_idx,line in enumerate(file): 
            split_line = line.split(" ")
            if line_idx == 0: 
                main_loss_val = split_line[3].strip("Loss:")
                mae = split_line[5].strip("MAE:")
                mse = split_line[7].strip("MSE:")
                mape = split_line[9].strip("MAPE:")
                loss_func_names = ['MAE', 'MSE', 'MAPE']
                for loss_idx,temp_loss_val in enumerate([mae,mse, mape]):
                    if temp_loss_val == main_loss_val:
                        loss_func_name = loss_func_names[loss_idx]
                print(f"mae [{mae}] mse [{mse}] mape [{mape}] loss_func_name [{loss_func_name}] loss_func_val[{main_loss_val}]")

            elif line_idx == 1:
                opt = split_line[1]
                learning_rate = split_line[4]
                dense_layer = split_line[8]
                print(f"opt [{opt}] lr [{learning_rate}] dense layer : [{dense_layer}]")
            elif line_idx == 2:
                future_candles = int(split_line[1])
                window_size = int(split_line[4])
                dropout = split_line[7]
                print(f"Candles [{future_candles}] window_size [{window_size}] dropout [{dropout}]")
            elif line_idx == 3:
                epochs = split_line[1]
                batch_size = split_line[4]
                run_time = (" ").join(split_line[7:11])

                print(f"Epochs [{epochs}] Batch_size [{batch_size}] run_time [{run_time}]")
            elif line_idx >= 4:
                #print("IDX: {} line: {}".format(line_idx,line))
                #print("IDX: {} line[28:-21]: {}".format(line_idx, line[28:-21]))
                if model_type == "LSTM":
                    if "input_layer_LSTM" in line:
                        input_layer_nodes = eval(line[28:-21])[-1]
                        #print("Tuple of interest" , eval(line[28:-21])[-1])
                    elif "1_h_layer_LSTM" in line: 
                        h_layer_1_nodes = eval(line[28:-21])[-1]
                        #print("Tuple of interest" ,eval(line[28:-21])[-1])
                    elif "2_h_layer_LSTM" in line: 
                        h_layer_2_nodes = eval(line[28:-21])[-1]
                        #print("Tuple of interest" ,eval(line[28:-21])[-1])
                    elif "3_h_layer_LSTM" in line: 
                        h_layer_3_nodes = eval(line[28:-21])[-1]
                        #print("Tuple of interest" ,eval(line[28:-21])[-1])
                    else:
                        pass

                if model_type == "CNN":
                    if "Stride" in line: 
                        activation_func = split_line[4]
                        print("activation_func: ", activation_func)
                    
                    elif "input_layer_CONV" in line:
                        input_layer_nodes = eval(line[28:-18])[-1]
                        #print("Tuple of interest" , eval(line[28:-21])[-1])
                    elif "1_h_layer_CONV" in line: 
                        h_layer_1_nodes = eval(line[28:-18])[-1]
                        #print("Tuple of interest" ,eval(line[28:-21])[-1])
                    elif "2_h_layer_CONV" in line: 
                        h_layer_2_nodes = eval(line[28:-18])[-1]
                        #print("Tuple of interest" ,eval(line[28:-21])[-1])
                    else:
                        pass                         
    
    
    if os.path.isfile(results_file):
        df = pd.read_csv(results_file)
        y_test,y_pred = df['y_test'] , df['y_pred'] 
        
        
    else :
        X_test, y_test = load_LSTM_test_data(model_type, window_size, future_candles)

        model_name = model_folder.lower() + ".h5"
        model = keras.models.load_model(all_models_folder + model_folder + "/" + model_name)

        y_pred = [y[0] for y in model.predict(X_test)]

        print("y_pred[:10] : ", y_pred[:10])
        print("y_test[:10] : ", y_test[:10])
        print()
        print("len(y_pred) :",len(y_pred))
        print("len(y_test) :",len(y_test))
        print("len(X_test) :",len(X_test))
        
        df = pd.DataFrame({'y_test' : y_test,
                           'y_pred':  y_pred})

        df.to_csv(results_file)

    if return_results:
        if model_type == "LSTM":
            return {"Model_number" : model_folder.replace("Model_", ""),
                    "MAE": mae,
                    "MSE": mse,
                    "MAPE": mape,
                   "Loss Function" : loss_func_name,
                   "Optimizer": opt,
                   "Learning Rate": learning_rate,
                   "Future_candles": future_candles,
                   "Window_size": window_size,
                   "Dropout": dropout,
                   "Epochs": epochs,
                   "Batch Size": batch_size,
                   "Runtime": run_time,
                   "Model type": model_type,
                   "Number of hidden layers": sum([bool(i) for i in [h_layer_1_nodes, h_layer_2_nodes, h_layer_3_nodes]])  ,
                   "input_layer_LSTM" :   input_layer_nodes,
                   "1_h_layer_LSTM_nodes" : h_layer_1_nodes,
                   "2_h_layer_LSTM_nodes" : h_layer_2_nodes,
                   "3_h_layer_LSTM_nodes" : h_layer_3_nodes,
                   "Dense Layer"          : dense_layer}
        
        if model_type == "CNN":
            return {"Model_number" : model_folder.replace("Model_", ""),
                    "MAE": mae,
                    "MSE": mse,
                    "MAPE": mape,
                   "Loss Function" : loss_func_name,
                   "Optimizer": opt,
                   "Learning Rate": learning_rate,
                   "Future_candles": future_candles,
                   "Window_size": window_size,
                   "Activation_function": activation_func,
                   "Epochs": epochs,
                   "Batch Size": batch_size,
                   "Runtime": run_time,
                   "Model type": model_type,
                   "Number of hidden layers": sum([bool(i) for i in [h_layer_1_nodes, h_layer_2_nodes, h_layer_3_nodes]])  ,
                   "input_layer_CONV": input_layer_nodes ,
                   "1_h_layer_CONV_filters": h_layer_1_nodes,
                   "2_h_layer_CONV_filters": h_layer_2_nodes,
                   "3_h_layer_CONV_filters": h_layer_3_nodes,
                    "Dense Layer"          : dense_layer}
            
    else:
        if return_dates:
            not_used_X_test, not_used_y_test, y_dates = load_LSTM_test_data(model_type, window_size, future_candles, return_dates = True)
            return y_test, y_pred, y_dates
        else:
            return y_test,y_pred 
    
#model_type = "LSTM"
#for model_type in ["LSTM", "CNN"]:
# for model_type in ["CNN", 'LSTM']:
#     all_models_folder = f"Saved_models/{model_type}_models/"
#     for model_folder in os.listdir(all_models_folder):
#         print(model_folder)
#         if "." in model_folder:
#             continue
#         y_test,y_pred = load_target_and_predictions(model_folder, model_type, return_results = False)
#         clear_output(wait=True)
#     print("\n"*2)

In [7]:
# Create a dataframe with all model performances 
# Each row should consist of the parameter attributes, along with their MAE,MSE,MAPE,MSEE



for model_type in ["CNN", "LSTM"]:
    if model_type in ["LSTM"]:
        df = pd.DataFrame(columns = ["Model_number",
                    "MAE",
                    "MSE",
                    "MAPE",
                   "Loss Function" ,
                   "Optimizer",
                   "Learning Rate",
                   "Future_candles",
                   "Window_size",
                   "Dropout",
                   "Epochs",
                   "Batch Size",
                   "Runtime",
                   "Model type" ,
                   "Number of hidden layers" , 
                   "input_layer_LSTM" ,
                   "1_h_layer_LSTM_nodes" ,
                   "2_h_layer_LSTM_nodes" ,
                   "3_h_layer_LSTM_nodes" ,
                    "Dense Layer"])
        
    if model_type == "CNN":
        df = pd.DataFrame(columns = ["Model_number",
                    "MAE",
                    "MSE",
                    "MAPE",
                   "Loss Function" ,
                   "Optimizer",
                   "Learning Rate",
                   "Future_candles",
                   "Window_size",
                   "Activation_function",
                   "Epochs",
                   "Batch Size",
                   "Runtime",
                   "Model type" ,
                   "Number of hidden layers" , 
                   "input_layer_CONV" ,
                   "1_h_layer_CONV_filters" ,
                   "2_h_layer_CONV_filters" ,
                   "3_h_layer_CONV_filters" ,
                    "Dense Layer"])
        
    all_models_folder = f"Saved_models/{model_type}_models/"
    results_file = all_models_folder + f"{model_type}_results.xlsx" 
    for model_folder in os.listdir(all_models_folder):              
        print(model_folder)
        if "." in model_folder:
            continue
        results_dict = load_target_and_predictions(model_folder, model_type, return_results = True)
        print(results_dict)
        df = df.append(results_dict, ignore_index= True)
        clear_output(wait=True)
        
    df = df.sort_values(by = "MSE").reset_index(drop = True)
    df.to_excel(results_file) #
    
    #top_runs = df.reset_index(drop=True).loc[:20,:] # Gets the TOP best
    
    #top_report = sv.analyze(top_runs)
    #top_report.show_html(results_file.replace(".xlsx","_TOP.html"))
    #all_report = sv.analyze(df)
    #all_report.show_html(results_file.replace("xlsx","html")) # Default arguments will generate to "SWEETVIZ_REPORT.html"
    
df

FileNotFoundError: [WinError 3] Den angivne sti blev ikke fundet: 'Saved_models/CNN_models/'

# FOREX  Trading Simulator



In [ ]:



def FOREX_simulator(model_folder, test_year, y_test , y_pred , y_dates,
                    future_candles , export_csv_results=False, view_single_setup = False, verbose = False):
    
    
    global all_models_folder
    
    temp_start = time.time()
    
    
    pred_mean = sum(y_pred)/len(y_pred)
    y_pred_std = statistics.stdev(y_pred)
    y_test_mean = sum(y_test)/len(y_test)
    
    if y_pred_std == 0:
        y_pred_std = 0.0000001
    
    spread = 4 # 4 pips (target changed to pips)
    
    threshold_1, threshold_2, threshold_3, threshold_4 =  y_pred_std , y_pred_std*2 , y_pred_std*3 ,  y_pred_std*4 
    print("")
    print("pred_mean: ", pred_mean)
    print(f"threshold_1: {threshold_1} , threshold_2: {threshold_2} , threshold_3: {threshold_3} , threshold_4: {threshold_4} ")
    
    
    trading_results_dict = {"Model_number" : int(model_folder.replace("Model_", "")),
                            "threshold" : 0,
                            "total_pips_gained": 0,
                            "total_pips_traded": 0,
                            "future_candles" : future_candles,
                            "pips_gained_pr_trade" : 0,
                            "wins": 0,
                            "losses": 0,
                            "total_trades": 0}
    
    forex_sim_df = pd.DataFrame(columns = ["Model_number" ,
                            "threshold",
                            "total_pips_gained",
                            "total_pips_traded",
                            "future_candles",
                            "pips_gained_pr_trade",
                            "wins",
                            "losses",
                            "total_trades"])
    
    threshold_results_dict = {f"threshold_{i+1}": trading_results_dict.copy() for i in range(4)}
    list_of_total_pips_gained = []
    list_of_trading_dates = []
    
    for thresh_idx, threshold in enumerate([threshold_1, threshold_2, threshold_3, threshold_4]):
        thresh_idx += 1
        
        if verbose:
            print(f"\nInitiating trading with threshold {thresh_idx}\n")
        
        total_pips_traded, total_pips_gained = [],[]
        dates_traded = ['2020-01-01T00:00']
        trade_active_timer = 0
        
        for idx,pred_price in enumerate(y_pred):
            if trade_active_timer > 0:
                trade_active_timer -= 1
                continue
            if (idx <= future_candles) or (idx >= (len(y_pred) - future_candles)):
                continue # trade simulation either to early or over.  
                
            y_price = y_test[idx]
            if pred_price > (pred_mean + threshold):
                # Buy
                trade_active_timer = future_candles
                pips_gained = y_test[idx] - (spread/2)
                
                # Round pips 
                pips_gained = round(pips_gained,2)
                
                total_pips_gained.append(pips_gained)
                total_pips_traded.append(abs(pips_gained))
                if verbose:
                    dates_traded.append(y_dates[idx])
                    print(f"BUY taken af idx: {idx}  date: {y_dates[idx]}  pips gained: {pips_gained} ")# Raw Target: {y_test[idx]} ")

            elif pred_price < (pred_mean - threshold):
                # Sell
                trade_active_timer = future_candles 
                pips_gained = (-y_test[idx] ) - (spread/2)
                
                # Round pips.
                pips_gained = round(pips_gained,2)
                
                total_pips_gained.append(pips_gained)
                total_pips_traded.append(abs(pips_gained))
                
                if verbose:
                    dates_traded.append(y_dates[idx])
                    print(f"SELL taken af idx: {idx}  date: {y_dates[idx]}  pips gained: {pips_gained} ") #Raw Target: {y_test[idx]} ")
                
            else: 
                # HODL
                pass
            
        if verbose:
            print("\n Total trades taken: {}  Wins: {}  Loss: {}  ".format(
                    len(total_pips_gained),
                    sum([1 if i > 0 else 0 for i in total_pips_gained]),
                    sum([1 if i <= 0 else 0 for i in total_pips_gained])))
            print(" pips gained: {}  pips traded: {}  pip_ratio: {}".format(
            sum(total_pips_gained), sum(total_pips_traded), sum(total_pips_gained)/sum(total_pips_traded)))
 
        if len(total_pips_gained) == 0:
            total_pips_gained.append(0)
            total_pips_traded.append(0.1) # to avoid division by 0 error.
             
        list_of_total_pips_gained.append(total_pips_gained)
        list_of_trading_dates.append(dates_traded)
        
        threshold_results_dict[f'threshold_{thresh_idx}']['total_pips_gained'] = sum(total_pips_gained)
        threshold_results_dict[f'threshold_{thresh_idx}']['total_pips_traded'] = sum(total_pips_traded)
        threshold_results_dict[f'threshold_{thresh_idx}']['wins']     = sum([1 if i > 0 else 0 for i in total_pips_gained])
        threshold_results_dict[f'threshold_{thresh_idx}']['losses']   = sum([1 if i < 0 else 0 for i in total_pips_gained]) 
        threshold_results_dict[f'threshold_{thresh_idx}']['total_trades'] = sum([1 for i in total_pips_gained])
        threshold_results_dict[f'threshold_{thresh_idx}']['pip_ratio'] = sum(total_pips_gained) / sum(total_pips_traded)
        threshold_results_dict[f'threshold_{thresh_idx}']['threshold'] = thresh_idx
        threshold_results_dict[f'threshold_{thresh_idx}']['threshold_value'] = threshold
        
        forex_sim_df = forex_sim_df.append(threshold_results_dict[f'threshold_{thresh_idx}'], ignore_index=True)
        
    if view_single_setup:
        best_threshold = -999999
        best_total_pips_gained = -999999
        best_list_from_total_pips_gained = []
        best_dates_traded = []
        
        for thresh_idx,total_pips_gained in enumerate(list_of_total_pips_gained):
            thresh_idx += 1
            if sum(total_pips_gained) > best_total_pips_gained:
                best_total_pips_gained = sum(total_pips_gained)
                best_threshold = thresh_idx
                best_list_from_total_pips_gained = total_pips_gained
                best_dates_traded = list_of_trading_dates[thresh_idx-1]
        
        # Plot 
        threshold_std = threshold_results_dict[f'threshold_{best_threshold}']['threshold_value']
        print(f"The best threshold was #{best_threshold}")   
        print(f" Threshold values:  LONG SIGNAL: {pred_mean+threshold_std}  SHORT SIGNAL: {pred_mean-threshold_std}")
        print("")
        plot_trading_simulation(best_list_from_total_pips_gained, best_dates_traded)   
        return None
        
    forex_sim_df.to_excel(all_models_folder + model_folder + "/FOREX_simulated_trades_info.xlsx")
        
    return threshold_results_dict



In [ ]:
#for model_type in ["LSTM", "CNN"]:
for model_type in ["LSTM"]:#, "CNN"]:
    
    forex_sim_df = pd.DataFrame(columns = ["Model_number" ,
                            "threshold_1_pips_gained" ,
                            "threshold_1_pip_ratio" ,
                            "threshold_2_pips_gained" ,
                            "threshold_2_pip_ratio" ,
                            "threshold_3_pips_gained" ,
                            "threshold_3_pip_ratio" ,
                            "threshold_4_pips_gained" ,
                            "threshold_4_pip_ratio",
                            "threshold_max_value"  ,
                            "threshold_max_pips_gained"  , 
                            "threshold_max_trades_won" ,
                            "threshold_max_trades_placed" ,
                            "threshold_max_pip_ratio",
                            "minimize_risk_max_profit" ])
    
    all_models_folder = f"Saved_models/{model_type}_models/"
    for _idx,model_folder in enumerate(os.listdir(all_models_folder)):
        if "." in model_folder:
            continue

        print(model_folder)

        model_dict = load_target_and_predictions(model_folder, model_type, return_results =True)
        y_test, y_pred = load_target_and_predictions(model_folder, model_type, return_results =False, return_dates= False)

        future_candles = model_dict['Future_candles']

        threshold_results_dict = FOREX_simulator(model_folder, 2020, y_test, y_pred, None ,future_candles, export_csv_results=False )
        
        best_threshold = 0
        best_threshold_pips_gained = -999999
        best_threshold_pip_ratio = -2
        best_trehshold_amount_of_trades = 0
        for thresh_idx in range(1,5):
            if threshold_results_dict[f'threshold_{thresh_idx}']['total_pips_gained'] > best_threshold_pips_gained:
                best_threshold_pip_ratio = threshold_results_dict[f'threshold_{thresh_idx}']['pip_ratio']
                best_threshold_pips_gained = threshold_results_dict[f'threshold_{thresh_idx}']['total_pips_gained']
                best_threshold = thresh_idx
        
        compact_threshold_results_dict = {"Model_number"    : threshold_results_dict[f'threshold_1']['Model_number'],
                                "threshold_1_pips_gained"   : threshold_results_dict[f'threshold_1']['total_pips_gained'],
                                "threshold_1_pip_ratio" : threshold_results_dict[f'threshold_1']['pip_ratio'],
                                "threshold_2_pips_gained"   : threshold_results_dict[f'threshold_2']['total_pips_gained'],
                                "threshold_2_pip_ratio" : threshold_results_dict[f'threshold_2']['pip_ratio'],
                                "threshold_3_pips_gained"   : threshold_results_dict[f'threshold_3']['total_pips_gained'],
                                "threshold_3_pip_ratio" : threshold_results_dict[f'threshold_3']['pip_ratio'],
                                "threshold_4_pips_gained"   : threshold_results_dict[f'threshold_4']['total_pips_gained'],
                                "threshold_4_pip_ratio" : threshold_results_dict[f'threshold_4']['pip_ratio'],
                                "threshold_max_value"                : best_threshold,
                                "threshold_max_pips_gained"          : best_threshold_pips_gained,
                                "threshold_max_trades_won"     : threshold_results_dict[f'threshold_{best_threshold}']['wins'],
                                "threshold_max_trades_placed"  : threshold_results_dict[f'threshold_{best_threshold}']['total_trades'],
                                "threshold_max_pip_ratio" : best_threshold_pip_ratio,
                                "minimize_risk_max_profit" : best_threshold_pip_ratio * best_threshold_pips_gained**2}

        forex_sim_df = forex_sim_df.append(compact_threshold_results_dict, ignore_index = True)
        
        clear_output(wait=True)
        
    results_df= pd.read_excel(all_models_folder + f"{model_type}_results.xlsx")
    final_df = pd.merge(results_df, forex_sim_df , on = "Model_number", how ="left")
    final_df.drop(columns = ['Unnamed: 0'], inplace = True)
    
    results_file = f"{model_type}_results_trading_sim.xlsx"
    
    final_df = final_df.sort_values(["minimize_risk_max_profit"], ascending = False)
    
    final_df.to_excel(all_models_folder + results_file)
    
    top_runs = final_df.reset_index(drop=True).loc[:20,:]
    
    top_report = sv.analyze(top_runs)
    top_report.show_html(all_models_folder + results_file.replace(".xlsx","_TOP.html"))
    
    all_report = sv.analyze(final_df)
    all_report.show_html(all_models_folder + results_file.replace("xlsx","html"))

        

# Base Model

In [ ]:
# Seeing results of specific models Prediction vs. Label for Perceptron

#df = pd.read_csv("Saved_models\\LSTM_models_1st_iter\\Model_1\\y_test_y_pred.csv")
#df = pd.read_csv("Saved_models\\LSTM_models_1st_iter\\Model_1\\y_test_y_pred.csv")
df = pd.read_csv("Saved_models\\Model_1969\\y_test_y_pred.csv")

predictions = df['y_pred']
y_test = df['y_test']

df = predictions_vs_actual(predictions, y_test)
plot_predictions(df)


In [ ]:
# Seeing results of specific models Trading performance
import pickle


model_type = "LSTM"
model_folder = "Model_1969"

all_models_folder = f"Saved_models/"

model_dict = load_target_and_predictions(model_folder, model_type, return_results =True)
y_test, y_pred, y_dates = load_target_and_predictions(model_folder, model_type, return_results =False, return_dates = True)

print(model_dict)

future_candles = model_dict['Future_candles']

threshold_results_dict = FOREX_simulator(model_folder, 2020, y_test, y_pred, y_dates ,future_candles, 
                                         export_csv_results=False , view_single_setup = True, verbose = True)


# Best performing LSTM model


In [ ]:
model_type = "LSTM"
model_folder = "Model_96"
hyper_param_iter = "1st"
all_models_folder = f"Saved_models\\{model_type}_models_{hyper_param_iter}_iter\\"


df = pd.read_csv(all_models_folder + model_folder+ "\\y_test_y_pred.csv")

predictions = df['y_pred']
y_test = df['y_test']

df = predictions_vs_actual(predictions, y_test)
plot_predictions(df)


model_dict = load_target_and_predictions(model_folder, model_type, return_results =True)
y_test, y_pred, y_dates = load_target_and_predictions(model_folder, model_type, return_results =False, return_dates = True)

print(model_dict)

future_candles = model_dict['Future_candles']

threshold_results_dict = FOREX_simulator(model_folder, 2020, y_test, y_pred, y_dates ,future_candles, 
                                         export_csv_results=False , view_single_setup = True, verbose = True)


In [ ]:
model_type = "CNN"
model_folder = "Model_87"
hyper_param_iter = "1st"
all_models_folder = f"Saved_models\\{model_type}_models_{hyper_param_iter}_iter\\"



df = pd.read_csv(all_models_folder + model_folder+ "\\y_test_y_pred.csv")

predictions = df['y_pred']
y_test = df['y_test']

df = predictions_vs_actual(predictions, y_test)
plot_predictions(df)


model_dict = load_target_and_predictions(model_folder, model_type, return_results =True)
y_test, y_pred, y_dates = load_target_and_predictions(model_folder, model_type, return_results =False, return_dates = True)

print(model_dict)

future_candles = model_dict['Future_candles']

threshold_results_dict = FOREX_simulator(model_folder, 2020, y_test, y_pred, y_dates ,future_candles, 
                                         export_csv_results=False , view_single_setup = True, verbose = True)



# Random Guessing formula

Given a spread of 4, and the normal distribution defined in the  random guessing should result in the following formula.

x = change in pips pips for trade \
n = trade placed
s = 4

$$
\mu = -0.02 \\
\sigma = 5.2987 \\
s = 4 \\
x_i ∼ N(\mu , \sigma); \quad i = 1,...,n \\
\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad \text{.} \\
Z∼N(0,1)\\
\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad \text{.} \\
Pips\ Gained = \left ( \sum_{i=1}^{n}{\left\{\begin{matrix}
-x_i ,& Z \geq  0\\ 
x_i ,& Z < 0
\end{matrix}\right. - \frac{s}{2} } \right ) \\
\quad\quad\quad\quad\quad\quad\quad \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\text{.} \\
\\
Pips\ Gained = n(\bar{x}-\frac{s}{2}) \\
Pip\ Ratio = \frac{n(\bar{x}-\frac{s}{2})}{\sum_{i=1}^{n}{|x_i|}} \\
\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\text{.} \\
\\
lim_{n\to\infty} \quad,\quad \bar{x} ∼ \mu \\
\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\text{.} \\
\\
Pip\ Ratio < 0 \\
Pips\ Gained = -\infty \\
$$
